In [ ]:
! pip install tokenizers
! pip install transformers
! pip install itertools

     |████████████████████████████████| 3.3 MB 5.6 MB/s 
     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 636 kB 49.8 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools


In [ ]:
# Mount Google Drive to this notebook
# The purpose is to allow your code to access to your files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/nlp_assignments/assignment1/

/content/drive/MyDrive/nlp_assignments/assignment1


In [ ]:
import pandas as pd
import numpy as np

##### Load the Dataset as pandas df and calculate average word size for each review in the training dataset. This will help us in setting the maximum word size for the bert tokenizer.

In [ ]:
train_df=pd.read_csv('/content/drive/My Drive/nlp_assignments/assignment1/imdbdataset/Train.csv')
test_df=pd.read_csv('/content/drive/My Drive/nlp_assignments/assignment1/imdbdataset/Test.csv')
val_df=pd.read_csv('/content/drive/My Drive/nlp_assignments/assignment1/imdbdataset/Valid.csv')
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)
# print(train_df[:5])
print(train_df['text'].size)
words=0
for i in range(train_df['text'].size):
  words+=len(train_df['text'][i].split())
print('avg words in training dataset: ',words/train_df['text'].size)
print(train_df[0:5])

(40000, 2)
(5000, 2)
(5000, 2)
40000
avg words in training dataset:  231.33925
                                                text  label
0  I grew up (b. 1965) watching and loving the Th...      0
1  When I put this movie in my DVD player, and sa...      0
2  Why do people who do not know what a particula...      0
3  Even though I have great interest in Biblical ...      0
4  Im a die hard Dads Army fan and nothing will e...      1


In [ ]:
from transformers import BertTokenizer, BertModel
encoder=BertTokenizer.from_pretrained('bert-base-cased')
# model = BertModel.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

##### Visualize encodings. This will be present in our dataloader getitem function.

In [ ]:
encodings=encoder(train_df['text'][1], add_special_tokens=True, padding='max_length', max_length=512,truncation=True)
encodings

{'input_ids': [101, 1332, 146, 1508, 1142, 2523, 1107, 1139, 4173, 1591, 117, 1105, 2068, 1205, 1114, 170, 1884, 2391, 1105, 1199, 13228, 117, 146, 1125, 1199, 11471, 119, 146, 1108, 4717, 1115, 1142, 2523, 1156, 4651, 1199, 1104, 1103, 2012, 118, 1827, 1104, 1103, 1148, 2523, 131, 138, 10732, 6758, 8794, 117, 1363, 8342, 1642, 117, 6548, 1490, 2641, 117, 6276, 3789, 1105, 170, 5642, 118, 3919, 5945, 119, 1252, 117, 1106, 1139, 10866, 117, 1136, 1251, 1104, 1142, 1110, 1106, 1129, 1276, 1107, 17793, 131, 22644, 112, 188, 11121, 119, 6467, 146, 2373, 1199, 3761, 1148, 117, 146, 1547, 1136, 1138, 1151, 1177, 1519, 1205, 119, 1109, 1378, 24950, 1209, 1129, 2002, 1106, 1343, 1150, 1138, 1562, 1103, 1148, 2523, 117, 1105, 1150, 4927, 1122, 3120, 1111, 1103, 1827, 3025, 119, 133, 9304, 120, 135, 133, 9304, 120, 135, 1332, 1103, 1148, 2741, 2691, 117, 1240, 1107, 1111, 170, 4900, 1191, 1128, 1198, 3015, 17793, 131, 22644, 112, 188, 11121, 1121, 1103, 3934, 118, 1692, 1120, 1240, 1469, 6581, 2

### Dataloader design:
##### For each item(__getitem__) we will return the encodings. The length operator will be standard.


In [ ]:
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

class imdbdataset(Dataset):
  def __init__(self, df, encoder):
    self.text=df['text']
    self.labels=df['label']
    self.encoder=encoder
  
  def __len__(self):
    return self.text.size

  def __getitem__(self,item):
    text=self.text[item]
    label=self.labels[item]
    encoding=encoder(text, add_special_tokens=True, padding='max_length', max_length=256,truncation=True, return_tensors="pt")
    return encoding['input_ids'].squeeze(0), encoding['attention_mask'].squeeze(0),encoding['token_type_ids'].squeeze(0),label
  





##### Launch the Training and validation datset and dataloader

In [ ]:
train_dataset=imdbdataset(train_df, encoder)
train_dataloader=DataLoader(train_dataset, batch_size=16)
val_dataset=imdbdataset(val_df, encoder)
val_dataloader=DataLoader(val_dataset, batch_size=16)

In [ ]:
inputids, attentionmasks, tokenids, label=next(iter(train_dataloader))
print(inputids)
print(attentionmasks)
print(label)
print(tokenids)

tensor([[ 101,  146, 2580,  ...,    0,    0,    0],
        [ 101, 1332,  146,  ..., 1210, 3426,  102],
        [ 101, 2009, 1202,  ...,    0,    0,    0],
        ...,
        [ 101, 6155, 3635,  ...,    0,    0,    0],
        [ 101,  138, 1544,  ...,    0,    0,    0],
        [ 101, 1409, 1128,  ...,    0,    0,    0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


###### Load the model. Here the sequence classification version of bert will be used. 

In [ ]:
# from transformers import BertForSequenceClassification
# model=BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)
from transformers import BertForSequenceClassification
from transformers import AdamW
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
d_model=model.to(device)

cuda:0


In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import get_scheduler

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
print(num_training_steps)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

5000


In [ ]:
!pip install sklearn
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
def compute_metrics(preds,labels):
  print(preds,labels)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  acc = accuracy_score(labels, preds)
  print("Precision = ", precision)
  print("recall = ", recall)
  print("F1 score = ", f1)
  print("accuracy = ", acc)

In [ ]:
import itertools
model.train()
for epoch in range(num_epochs):
  rev_predictions=[]
  rev_labels=[]
  iter=0
  for input_ids, token_ids, attentionmasks,labels in train_dataloader:
    iter+=1
    d_inputids=input_ids.to(device)
    d_tokenids=token_ids.to(device)
    d_attentionmasks=attentionmasks.to(device)
    d_labels=labels.to(device)
    outputs = model(d_inputids, d_tokenids, d_attentionmasks, labels=d_labels)
    loss=outputs.loss
    if (iter%99==0):
      print(iter)
      print(loss)
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
  print("1 epoch terminated. Evaluating on validation....")
  model.eval()
  for input_ids, token_ids, attentionmasks,labels in itertools.islice(val_dataloader,100):
    d_inputids=input_ids.to(device)
    d_tokenids=token_ids.to(device)
    d_attentionmasks=attentionmasks.to(device)
    d_labels=labels.to(device)
    with torch.no_grad():
      outputs = model(d_inputids, d_tokenids, d_attentionmasks, labels=d_labels)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    rev_predictions.extend(predictions)
    rev_labels.extend(d_labels)
  compute_metrics(torch.stack(rev_predictions).cpu(), torch.stack(rev_labels).cpu())  
print(rev_labels,rev_predictions)
torch.save(model.state_dict(),'/content/drive/My Drive/nlp_assignments/assignment1/pytorch_model.bin')
model.config.to_json_file('/content/drive/My Drive/nlp_assignments/assignment1/config.json')



99
tensor(0.4155, device='cuda:0', grad_fn=<NllLossBackward>)
198
tensor(0.1302, device='cuda:0', grad_fn=<NllLossBackward>)
297
tensor(0.4737, device='cuda:0', grad_fn=<NllLossBackward>)
396
tensor(0.1300, device='cuda:0', grad_fn=<NllLossBackward>)
495
tensor(0.1956, device='cuda:0', grad_fn=<NllLossBackward>)
594
tensor(0.5038, device='cuda:0', grad_fn=<NllLossBackward>)
693
tensor(0.2507, device='cuda:0', grad_fn=<NllLossBackward>)
792
tensor(0.4452, device='cuda:0', grad_fn=<NllLossBackward>)
891
tensor(0.6277, device='cuda:0', grad_fn=<NllLossBackward>)
990
tensor(0.1390, device='cuda:0', grad_fn=<NllLossBackward>)
1089
tensor(0.1404, device='cuda:0', grad_fn=<NllLossBackward>)
1188
tensor(0.2277, device='cuda:0', grad_fn=<NllLossBackward>)
1287
tensor(0.4439, device='cuda:0', grad_fn=<NllLossBackward>)
1386
tensor(0.1085, device='cuda:0', grad_fn=<NllLossBackward>)
1485
tensor(0.2016, device='cuda:0', grad_fn=<NllLossBackward>)
1584
tensor(0.1900, device='cuda:0', grad_fn=<NllLo

NameError: ignored

In [ ]:
import itertools
model.eval()
rev_predictions=[]
rev_labels=[]
for input_ids, token_ids, attentionmasks,labels in itertools.islice(val_dataloader,100):
  print(i)
  d_inputids=input_ids.to(device)
  d_tokenids=token_ids.to(device)
  d_attentionmasks=attentionmasks.to(device)
  d_labels=labels.to(device)
  with torch.no_grad():
    outputs = model(d_inputids, d_tokenids, d_attentionmasks, labels=d_labels)
  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)
  rev_predictions.extend(predictions)
  rev_labels.extend(d_labels)
# print(rev_predictions,rev_labels)
compute_metrics(torch.stack(rev_predictions).cpu(), torch.stack(rev_labels).cpu())  
# print(rev_labels,rev_predictions)

39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
39999
tensor([0, 0, 0,  ..., 0, 1, 0]) tensor([0, 0, 0,  ..., 0, 1, 0])
Precision =  0.9239263803680982
recall =  0.9261992619926199
F1 score =  0.925061425061425
accuracy =  0.92375


In [ ]:
model.config.to_json_file('/content/drive/My Drive/nlp_assignments/assignment1/config.json')


###### Check accuracy and F1 score on the test dataset

In [ ]:
test_dataset=imdbdataset(test_df, encoder)
test_dataloader=DataLoader(test_dataset, batch_size=16)
num_iters=16
load_model= BertForSequenceClassification.from_pretrained('/content/drive/My Drive/nlp_assignments/assignment1/')
load_model.to(device)
import itertools
load_model.eval()
rev_predictions=[]
rev_labels=[]
for iters in range(num_iters):
  for input_ids, token_ids, attentionmasks,labels in test_dataloader:
    d_inputids=input_ids.to(device)
    d_tokenids=token_ids.to(device)
    d_attentionmasks=attentionmasks.to(device)
    d_labels=labels.to(device)
    with torch.no_grad():
      outputs = load_model(d_inputids, d_tokenids, d_attentionmasks, labels=d_labels)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    rev_predictions.extend(predictions)
    rev_labels.extend(d_labels)
compute_metrics(torch.stack(rev_predictions).cpu(), torch.stack(rev_labels).cpu())  

tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0, 0, 0,  ..., 0, 0, 0])
Precision =  0.9159170903402425
recall =  0.9349301397205588
F1 score =  0.9253259581193203
accuracy =  0.9244
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0, 0, 0,  ..., 0, 0, 0])
Precision =  0.9159170903402425
recall =  0.9349301397205588
F1 score =  0.9253259581193203
accuracy =  0.9244
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0, 0, 0,  ..., 0, 0, 0])
Precision =  0.9159170903402425
recall =  0.9349301397205588
F1 score =  0.9253259581193203
accuracy =  0.9244
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0, 0, 0,  ..., 0, 0, 0])
Precision =  0.9159170903402425
recall =  0.9349301397205588
F1 score =  0.9253259581193203
accuracy =  0.9244
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0, 0, 0,  ..., 0, 0, 0])
Precision =  0.9159170903402425
recall =  0.9349301397205588
F1 score =  0.9253259581193203
accuracy =  0.9244
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0, 0, 0,  ..., 0, 0, 0])
Precision =  0.9159170903402425
recall =  0.93493

In [ ]:
!apt-get -qq install texlive texlive-xetex texlive-latex-extra pandoc
!pip install --quiet pypandoc

Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package fonts-droid-fallback.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../00-fonts-droid-fallback_1%3a6.0.1r16-1.1_all.deb ...
Unpacking fonts-droid-fallback (1:6.0.1r16-1.1) ...
Selecting previously unselected package fonts-lato.
Preparing to unpack .../01-fonts-lato_2.0-2_all.deb ...
Unpacking fonts-lato (2.0-2) ...
Selecting previously unselected package poppler-data.
Preparing to unpack .../02-poppler-data_0.4.8-2_all.deb ...
Unpacking poppler-data (0.4.8-2) ...
Selecting previously unselected package tex-common.
Preparing to unpack .../03-tex-common_6.09_all.deb ...
Unpacking tex-common (6.09) ...
Selecting previously unselected package fonts-lmodern.
Preparing to unpack .../04-fonts-lmodern_2.004.5-3_all.deb ...
Unpacking fonts-lmodern (2.004.5-3) ...
Selecting previously unselected package fonts-noto-mono.
Preparing to unpack .

In [ ]:
!jupyter nbconvert --to PDF "/content/drive/MyDrive/nlp_assignments/assignment1/Assignment1_apoorvgarg.ipynb"

[NbConvertApp] Converting notebook /content/drive/MyDrive/nlp_assignments/assignment1/Assignment1_apoorvgarg.ipynb to PDF
[NbConvertApp] Writing 172311 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 76704 bytes to /content/drive/MyDrive/nlp_assignments/assignment1/Assignment1_apoorvgarg.pdf
